In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

#设置随机种子
torch.manual_seed(12046)
#数据准备（CIFAR10专用）
transform_train=transforms.Compose([
    transforms.RandomCrop(32,padding=4),#随意裁剪图像
    transforms.RandomHorizontalFlip(),#随机水平翻转
    transforms.ToTensor(),#转为张量
    transforms.Normalize((0.4914,0.4822,0.4465),(0.2470,0.2435,0.2616)),#归一化，这里使用的是针对这个数据集的均值和标准差
])

transform_test=transforms.Compose([#测试集的预处理
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),(0.2470,0.2435,0.2616)),
])

#加载数据集，并且将预处理部分应用进来
train_set=datasets.CIFAR10(root='./data',train=True,download=True,transform=transform_train)
test_set=datasets.CIFAR10(root='./data',train=False,download=True,transform=transform_test)
#划分验证集，比例为9：1
train_size=int(0.9*len(train_set))
val_size=len(train_set)-train_size
train_set,val_set=torch.utils.data.random_split(train_set,[train_size,val_size])
# 创建DataLoader
batch_size=128#每个批次数据量为128
train_loader=DataLoader(train_set,batch_size=batch_size,shuffle=True,num_workers=2)
val_loader=DataLoader(val_set,batch_size=batch_size,shuffle=False,num_workers=2)
test_loader=DataLoader(test_set,batch_size=batch_size,shuffle=False,num_workers=2)

# 残差块
class ResidualBlock(nn.Module):
    def __init__(self,in_channel,out_channel,stride=1):
        super().__init__()
        self.conv1=nn.Conv2d(in_channel,out_channel,kernel_size=3,stride=stride,padding=1,bias=False)
        self.bn1=nn.BatchNorm2d(out_channel)
        self.conv2=nn.Conv2d(out_channel,out_channel,kernel_size=3,stride=1,padding=1,bias=False)
        self.bn2=nn.BatchNorm2d(out_channel)
        #下采样模块
        self.downsample=nn.Sequential()
        if stride!=1 or in_channel!=out_channel:
            self.downsample=nn.Sequential(
                nn.Conv2d(in_channel,out_channel,kernel_size=1,stride=stride,bias=False),
                nn.BatchNorm2d(out_channel)
            )

    def forward(self, x):
        identity=self.downsample(x)
        out=F.relu(self.bn1(self.conv1(x)))
        out=self.bn2(self.conv2(out))
        out+=identity
        out=F.relu(out)
        return out

#CIFAR10的ResNet
class ResNet_CIFAR10(nn.Module):
    def __init__(self):
        super().__init__()
        #初始层
        self.conv1=nn.Conv2d(3, 64, kernel_size=3,stride=1,padding=1,bias=False)
        self.bn1=nn.BatchNorm2d(64)

        # 残差块组4个阶段
        self.layer1=self._make_layer(64,64,stride=1)
        self.layer2=self._make_layer(64,128,stride=2)
        self.layer3=self._make_layer(128,256,stride=2)
        self.layer4=self._make_layer(256,512,stride=2)

        # 分类头
        self.avgpool=nn.AdaptiveAvgPool2d((1,1))
        self.fc=nn.Linear(512,10)

    def _make_layer(self,in_channels,out_channels,stride):
        return nn.Sequential(
            ResidualBlock(in_channels,out_channels,stride),
            ResidualBlock(out_channels,out_channels,stride=1)
        )

    def forward(self,x):
        x=F.relu(self.bn1(self.conv1(x)))  #[B,64,32,32]
        x=self.layer1(x)  #[B,64,32,32]
        x=self.layer2(x)  #[B,128,16,16]
        x=self.layer3(x)  # [B,256,8,8]
        x=self.layer4(x)  # [B,512,4,4]
        x =self.avgpool(x)  #[B,512,1,1]
        x=torch.flatten(x,1)  #[B,512]
        x=self.fc(x)  #[B,10]
        return x

# 训练工具函数
def estimate_loss(model,data_loader,eval_iters=10):
    model.eval()
    losses=[]
    accuracies=[]
    for i,(inputs,labels) in enumerate(data_loader):
        if i>=eval_iters: break
        with torch.no_grad():
            logits=model(inputs)
            loss=F.cross_entropy(logits,labels)
            _,predicted=torch.max(logits,1)
            acc=(predicted==labels).float().mean()
        losses.append(loss.item())
        accuracies.append(acc.item())
    model.train()
    return {
        'loss': sum(losses)/len(losses),
        'accuracy': sum(accuracies)/len(accuracies)
    }

def train(model,optimizer,epochs=10):
    for epoch in range(epochs):
        model.train()
        for inputs,labels in train_loader:
            optimizer.zero_grad()
            logits=model(inputs)
            loss=F.cross_entropy(logits,labels)
            loss.backward()
            optimizer.step()
        # 评估
        train_stats=estimate_loss(model,train_loader)
        val_stats =estimate_loss(model,val_loader)
        test_stats=estimate_loss(model,test_loader)
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Train Loss: {train_stats['loss']:.4f} | Val Loss: {val_stats['loss']:.4f} | Test Loss: {test_stats['loss']:.4f}")
        print(f"Train Acc: {train_stats['accuracy']:.4f} | Val Acc: {val_stats['accuracy']:.4f} | Test Acc: {test_stats['accuracy']:.4f}")
        print("-"*50)

#训练模型
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=ResNet_CIFAR10().to(device)
optimizer=optim.Adam(model.parameters(),lr=0.001,weight_decay=1e-4)
# 将DataLoader移到设备（如果使用GPU）
train_loader=DataLoader(train_set,batch_size=batch_size,shuffle=True,num_workers=2,pin_memory=True)
val_loader=DataLoader(val_set,batch_size=batch_size,shuffle=False,num_workers=2,pin_memory=True)
test_loader=DataLoader(test_set,batch_size=batch_size,shuffle=False,num_workers=2,pin_memory=True)
# 开始训练
train(model,optimizer,epochs=20)

Epoch 1/20
Train Loss: 1.4973 | Val Loss: 1.5177 | Test Loss: 1.5295
Train Acc: 0.4766 | Val Acc: 0.4711 | Test Acc: 0.4766
--------------------------------------------------
Epoch 2/20
Train Loss: 1.1512 | Val Loss: 1.1509 | Test Loss: 1.2242
Train Acc: 0.6094 | Val Acc: 0.6062 | Test Acc: 0.5844
--------------------------------------------------
Epoch 3/20
Train Loss: 0.8376 | Val Loss: 0.8531 | Test Loss: 0.9066
Train Acc: 0.7055 | Val Acc: 0.7031 | Test Acc: 0.6758
--------------------------------------------------
Epoch 4/20
Train Loss: 0.8637 | Val Loss: 0.9130 | Test Loss: 0.8641
Train Acc: 0.7250 | Val Acc: 0.6937 | Test Acc: 0.7242
--------------------------------------------------
Epoch 5/20
Train Loss: 0.6087 | Val Loss: 0.6669 | Test Loss: 0.6659
Train Acc: 0.7992 | Val Acc: 0.7922 | Test Acc: 0.7922
--------------------------------------------------
Epoch 6/20
Train Loss: 0.6765 | Val Loss: 0.7283 | Test Loss: 0.8303
Train Acc: 0.7602 | Val Acc: 0.7445 | Test Acc: 0.7211
-